Load expressions
----------------

### imports ###

A hack to use local version o yspecies for debugging

In [1]:
from pathlib import Path
import sys
import inspect
debug_local = True

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)

extending pathes with local yspecies
['/data/sources/yspecies/yspecies', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/yspecies-0.1.5-py3.8.egg', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [2]:
(Path("..") / "yspecies").resolve()

PosixPath('/data/sources/yspecies/yspecies')

In [3]:
from typing import *
from yspecies import *
from yspecies.enums import *
from yspecies.dataset import *
from yspecies.misc import *
from yspecies.workflow import *

In [4]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
import pandas as pd
import shap
from pprint import pprint
import random
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import lightgbm as lgb
from scipy.stats import kendalltau
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, recall_score, precision_score, f1_score

In [6]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [7]:
number_of_bootstraps = 5 # this sets global setting of which how many bootstraps to use

lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'max_leaves': 20,
    'max_depth': 3,
    'learning_rate': 0.07,
    'feature_fraction': 0.8,
    'bagging_fraction': 1,
    'min_data_in_leaf': 6,
    'lambda_l1': 0.9,
    'lambda_l2': 0.9,
    "verbose": -1
}

### Loading data ###

In [8]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [9]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(445, 12243)",12243,39,445,"(67996, 3)","(40, 18)"


In [190]:

@dataclass
class SelectedFeatures:

    samples: List[str] = None
    species: List[str] = None
    genes: List[str] = None #if None = takes all genes
    to_predict: str = "lifespan"
        
    @property
    def y_name(self):
        return f"Y_{self.to_predict}"

    content = None #content feature to

    def with_content(self, content) -> 'SelectedFeatures':
        self.content = content
        return self

    
    def _wrap_html_(self, content):
        if content is None:
            return ""
        elif isinstance(content, pd.DataFrame):
            if(content.shape[1]>100):
                return show(content, 10, 10)._repr_html_()
            else:
                return content.head(10)._repr_html_()
        else:
            return content._repr_html_()
        
    def _repr_html_(self):
        return f"<table border='2'>" \
               f"<caption> Selected feature columns <caption>" \
               f"<tr><th>Samples metadata</th><th>Species metadata</th><th>Genes</th><th>Predict label</th></tr>" \
               f"<tr><td>{str(self.samples)}</td><td>{str(self.species)}</td><td>{'all' if self.genes is None else str(self.genes)}</td><td>{str(self.to_predict)}</td></tr>" \
               f"</table>{self._wrap_html_(self.content)}"

In [204]:
@dataclass
class DataExtractor(TransformerMixin):
    features: SelectedFeatures
    
    def fit(self, X, y=None) -> 'DataExtractor':
        return self
    
    def transform(self, data: ExpressionDataset) -> pd.DataFrame:
        samples = data.extended_samples(self.features.samples, self.features.species)
        exp = data.expressions if self.features.genes is None else data.expressions[self.features.genes]
        X: pd.DataFrame = samples.join(exp)
        content = data.get_label(self.features.to_predict).join(X).rename(columns={self.features.to_predict: self.features.y_name})
        return self.features.with_content(content)


In [213]:
@dataclass
class DataPartitioner(TransformerMixin):
    '''
    Partitions the data according to sorted stratification
    '''
    folds: int = 5

    def fit(self, X, y=None) -> 'DataExtractor':
        return self

    def transform(self, selected: SelectedFeatures) -> ExpressionPartitions:
        '''

        :param data: ExpressionDataset
        :param k: number of k-folds in sorted stratification
        :return: partitions
        '''
        assert isinstance(selected.content, pd.DataFrame), "Should contain extracted Pandas DataFrame with X and Y"
        return self.sorted_stratification(selected.content, selected, self.folds)

    def sorted_stratification(self, df: pd.DataFrame, features: SelectedFeatures, k: int) -> ExpressionPartitions:
        '''
        paritions the data according to sorted_stratificaiton algofirthm
        :param X:
        :param Y:
        :param k:
        :return:
        '''
        X = df.sort_values(by=[features.y_name])
        partition_indexes = [[] for i in range(k)]
        i = 0
        index_of_sample = 0

        while i < (int(len(X)/k)):
            for j in range(k):
                partition_indexes[j].append((i*k)+j)
                index_of_sample = (i*k)+j
            i += 1

        index_of_sample += 1
        i = 0
        while index_of_sample < len(X):
            partition_indexes[i].append(index_of_sample)
            index_of_sample += 1
            i += 1

        X_sorted = X.drop([features.y_name], axis=1)
        Y_sorted = X[[features.y_name]].rename(columns={features.y_name: features.to_predict})

        x_partitions = []
        y_partitions = []
        for pindex in partition_indexes:
            x_partitions.append(X_sorted.iloc[pindex])
            y_partitions.append(Y_sorted[features.to_predict].iloc[pindex])

        return ExpressionPartitions(features, X_sorted, Y_sorted, x_partitions, y_partitions, k)

In [222]:
import shap
from scipy.stats import kendalltau


@dataclass
class FeatureResults:
    weighted_features: List
    stable_shap_values: List

@dataclass
class ModelFactory:

    default_parameters: Dict = field(default_factory = lambda : {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'l2', 'l1'},
        'max_leaves': 20,
        'max_depth': 3,
        'learning_rate': 0.07,
        'feature_fraction': 0.8,
        'bagging_fraction': 1,
        'min_data_in_leaf': 6,
        'lambda_l1': 0.9,
        'lambda_l2': 0.9,
        "verbose": -1
    })


    def regression_model(self, X_train, X_test, y_train, y_test, categorical=None, parameters: dict = None): #, categorical):
        categorical = categorical if isinstance(categorical, List) or categorical is None else [categorical]
        parameters = self.default_parameters if parameters is None else parameters
        lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical)
        lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
        evals_result = {}
        gbm = lgb.train(parameters,
                        lgb_train,
                        num_boost_round=500,
                        valid_sets=lgb_eval,
                        evals_result=evals_result,
                        verbose_eval=1000,
                        early_stopping_rounds=7)
        return gbm

@dataclass
class FeatureAnalyzer(TransformerMixin):
    '''
    Class that gets partioner and model factory and selects best features.
    TODO: rewrite everything to Pipeline
    '''

    model_factory: ModelFactory
    bootstraps: int = 5

    def fit(self, X, y=None) -> 'DataExtractor':
        return self

    def transform(self, partitions: ExpressionPartitions) -> FeatureResults:
        weight_of_features = []
        shap_values_out_of_fold = [[0 for i in range(len(partitions.X.values[0]))] for z in range(len(partitions.X))]
        #interaction_values_out_of_fold = [[[0 for i in range(len(X.values[0]))] for i in range(len(X.values[0]))] for z in range(len(X))]
        out_of_folds_metrics = [0, 0, 0]

        for i in range(self.bootstraps):

            X_test = partitions.x_partitions[i]
            y_test = partitions.y_partitions[i]

            X_train = pd.concat(partitions.x_partitions[:i] + partitions.x_partitions[i+1:])
            y_train = np.concatenate(partitions.y_partitions[:i] + partitions.y_partitions[i+1:], axis=0)

            # get trained model and record accuracy metrics
            model = self.model_factory.regression_model(X_train, X_test, y_train, y_test)#, index_of_categorical)
            #out_of_folds_metrics = add_metrics(out_of_folds_metrics, model, X_test, y_test)

            weight_of_features.append(model.feature_importance(importance_type='gain'))

            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(partitions.X)
            #interaction_values = explainer.shap_interaction_values(X)
            shap_values_out_of_fold = np.add(shap_values_out_of_fold, shap_values)
            #interaction_values_out_of_fold = np.add(interaction_values_out_of_fold, interaction_values)

        # print average metrics results
        #print('Accuracy of predicting ' + label_to_predict, np.divide(out_of_folds_metrics, self.bootstraps))

        # calculate shap values out of fold
        shap_values_out_of_fold = shap_values_out_of_fold / self.bootstraps
        shap_values_transposed = shap_values_out_of_fold.T
        X_transposed = partitions.X.T.values

        # get features that have stable weight across bootstraps
        output_features_by_weight = []
        for i, index_of_col in enumerate(weight_of_features[0]):
            cols = []
            for sample in weight_of_features:
                cols.append(sample[i])
            non_zero_cols = 0
            for col in cols:
                if col != 0:
                    non_zero_cols += 1
            if non_zero_cols == self.bootstraps:
                if 'ENSG' in partitions.X.columns[i]:
                    output_features_by_weight.append({
                        'ids': partitions.X.columns[i],
                        'gain_score_to_'+partitions.label_to_predict: np.mean(cols),
                        'name': partitions.X.columns[i], #ensemble_data.gene_name_of_gene_id(X.columns[i]),
                        'kendall_tau_to_'+partitions.label_to_predict: kendalltau(shap_values_transposed[i], X_transposed[i], nan_policy='omit')[0]
                    })

        #output_features_by_weight = sorted(output_features_by_weight, key=lambda k: k['score'], reverse=True)

        return FeatureResults(output_features_by_weight, shap_values_out_of_fold)

In [223]:
from sklearn.pipeline import Pipeline
selection = SelectedFeatures([],[], to_predict = "lifespan")
pipe = Pipeline([
    ('extractor', DataExtractor(selection)), 
    ("partitioner", DataPartitioner()), 
    ("shap_computation", FeatureAnalyzer(ModelFactory()))]
)

In [224]:
d = pipe.fit_transform(data)
d

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: species

In [10]:
from yspecies.shap_selection import *
partitioner = DataPartitioner(SelectedFeatures([],[], to_predict = "lifespan"))
factory =  ModelFactory()

In [41]:
i = 0
for j in range(0, len(partitions.x_partitions)):
    partitions.x_partitions[j] = partitions.x_partitions[j].drop(columns=["species"], errors='ignore')
    
X_test = partitions.x_partitions[i]
y_test = partitions.y_partitions[i]

X_train = pd.concat(partitions.x_partitions[:i] + partitions.x_partitions[i+1:])
y_train = np.concatenate(partitions.y_partitions[:i] + partitions.y_partitions[i+1:], axis=0)

In [51]:
show(X_test, 10)

,ENSG00000139990,ENSG00000073921,ENSG00000139687,ENSG00000119977,ENSG00000242866,ENSG00000135506,ENSG00000162426,ENSG00000165995,ENSG00000073756,ENSG00000138050
run,,,,,,,,,,
ERR1990034,18.069,362.355,66.914,8.090,0.000,114.738,0.125,2.260,1.170,4.068
ERR3350030,5.975,28.073,1.280,0.858,0.000,87.180,0.000,0.000,0.000,2.087
ERR1990033,27.991,153.457,25.057,3.140,0.000,30.831,0.196,100.551,3.070,4.349


In [54]:
X_test["ENSG00000139990"].iloc[10:20]

run
SRR3144817   19.794
SRR5520664    5.125
SRR3144815   20.444
SRR5520667    3.498
SRR3468369    3.352
SRR636908    16.833
SRR636857    24.597
SRR3468375    9.049
SRR2925249    8.780
SRR2925253    6.966
Name: ENSG00000139990, dtype: float64

In [43]:
factory.regression_model(X_train, X_test, y_train, y_test)#, index_of_categorical)

TypeError: Wrong type(ndarray) for label.
It should be list, numpy 1-D array or pandas Series

In [12]:
selected = analyzer.select_features(data)
selected

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: species

In [12]:
def calculate_metrics(prediction, ground_truth):
     return {
            'R2': r2_score(ground_truth, prediction),
            'MSE': mean_squared_error(ground_truth, prediction),
            'MAE': mean_absolute_error(ground_truth, prediction),
     }
    
def encode_tissues(dataframe):
    le.fit(dataframe['tissue'].values)
    tissues_encoded = le.transform(dataframe['tissue'].values)
    dataframe['tissue_encoded'] = tissues_encoded
    
    return dataframe
    
    
def split_to_X_and_Y(dataframe, label_to_predict):
    if 'tissue' in dataframe.columns:
        X = dataframe.drop([label_to_predict, 'tissue'], axis=1)
        Y = dataframe[label_to_predict].values
        index_of_categorical_feature = list(X.columns).index('tissue_encoded')
    else:
        X = dataframe.drop([label_to_predict], axis=1)
        Y = dataframe[label_to_predict].values
        index_of_categorical_feature = None

    return X, X.values, Y, index_of_categorical_feature
    
    
def get_predictions(label_to_predict, ids=None):
    species_data = pd.read_csv('cross_species_df_merged.csv', low_memory=False)
    
    # remove other features (redundant and those that correlate with target)
    cols_to_delete = []
    for column in list(species_data.columns):
        if ids:
            if column not in ids and column not in ['tissue', label_to_predict]:
                cols_to_delete.append(column)
        else:
            if 'ENSG' not in column and column not in ['tissue', label_to_predict]:
                cols_to_delete.append(column)    
            
    species_data = species_data.drop(cols_to_delete, axis=1) 
    
    species_data = species_data[(~pd.isnull(species_data[label_to_predict]))] # select only row where target is set
    species_data = species_data.dropna(axis=1, thresh=int(len(species_data)*0.9)) # remove all genes where percentage of NaN > 10%
    species_data = species_data[species_data['tissue'].isin(['Lung', 'Liver', 'Kidney', 'Brain', 'Heart'])] # remove underrepresented tissues
    species_data = encode_tissues(species_data)
    
    print('Number of samples', len(species_data))
    print('Number of genes', len(species_data.columns))
    
    feature_df, X, Y, index_of_categorical = split_to_X_and_Y(species_data, label_to_predict)
    
    object_from_training = calculate_stable_shap_values(feature_df, Y, index_of_categorical, label_to_predict)
    features_weighted = object_from_training['list_of_weighted_features']
    shap_values = object_from_training['stable_shap_values']
    
    return shap_values, feature_df, features_weighted

### Get list of selected genes for each variable

In [13]:
lifespan_weighted_features = []
lifespan_shap_values = []
lifespan_dataframes = []

for label in ['gestation_days', 'max_lifespan', 'mass_g', 'temperature_celsius', 'metabolic_rate', 'mtGC']:
    shap_values, feature_df, weighted_features = get_predictions(label)
    lifespan_weighted_features += weighted_features
    lifespan_shap_values.append(shap_values)
    lifespan_dataframes.append(feature_df)
    

FileNotFoundError: [Errno 2] File cross_species_df_merged.csv does not exist: 'cross_species_df_merged.csv'